# Analyzing results from hyperparameter tuning
In this example we'll run a hyperparameter tuning experiment and analyze the results.

What we'll cover:
- How to load data from an Sklearn example dataset
- How to initialize an XGBoost trainer
- How to define a search space for regular XGBoost parameters and for data preprocessors
- How to fetch the best obtained result from the tuning run
- How to fetch a dataframe to do further analysis on the results

First we start with some imports. We'll use the [Covertype dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn-datasets-fetch-covtype) provided from sklearn to train a multiclass classification task using XGBoost.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_covtype

import ray
from ray import tune
from ray.ml import RunConfig
from ray.ml.preprocessors import StandardScaler
from ray.ml.train.integrations.xgboost import XGBoostTrainer
from ray.tune.tune_config import TuneConfig
from ray.tune.tuner import Tuner

We'll define a utility function to create a Ray Dataset from the Sklearn dataset. We expect the target column to be in the dataframe, so we'll add it to the dataframe manually.
We don't do any test/validation splitting here - pelase check out our other examples for this.

In [2]:
def get_training_data() -> ray.data.Dataset:
    data_raw = fetch_covtype()
    df = pd.DataFrame(data_raw["data"], columns=data_raw["feature_names"])
    df["target"] = data_raw["target"]
    return ray.data.from_pandas(df)


train_dataset = get_training_data()

2022-05-09 17:25:30,768	INFO services.py:1484 -- View the Ray dashboard at http://127.0.0.1:8265


Let's take a look at the schema here:

In [3]:
print(train_dataset)

Dataset(num_blocks=1, num_rows=581012, schema={Elevation: float64, Aspect: float64, Slope: float64, Horizontal_Distance_To_Hydrology: float64, Vertical_Distance_To_Hydrology: float64, Horizontal_Distance_To_Roadways: float64, Hillshade_9am: float64, Hillshade_Noon: float64, Hillshade_3pm: float64, Horizontal_Distance_To_Fire_Points: float64, Wilderness_Area_0: float64, Wilderness_Area_1: float64, Wilderness_Area_2: float64, Wilderness_Area_3: float64, Soil_Type_0: float64, Soil_Type_1: float64, Soil_Type_2: float64, Soil_Type_3: float64, Soil_Type_4: float64, Soil_Type_5: float64, Soil_Type_6: float64, Soil_Type_7: float64, Soil_Type_8: float64, Soil_Type_9: float64, Soil_Type_10: float64, Soil_Type_11: float64, Soil_Type_12: float64, Soil_Type_13: float64, Soil_Type_14: float64, Soil_Type_15: float64, Soil_Type_16: float64, Soil_Type_17: float64, Soil_Type_18: float64, Soil_Type_19: float64, Soil_Type_20: float64, Soil_Type_21: float64, Soil_Type_22: float64, Soil_Type_23: float64, So

Ray AIR allows us to define and tune preprocessors. For instance, we may want to decide if we should scale the "Wilderness area" parameters or the "Soil type" parameters. For both, we'll just try standard scalers:

In [4]:
# Let's tune the preprocessors
prep_v1 = StandardScaler(columns=[f"Wilderness_Area_{i}" for i in range(4)])
prep_v2 = StandardScaler(columns=[f"Soil_Type_{i}" for i in range(40)])

Since we'll be training a multiclass prediction model, we have to pass some information to XGBoost. For instance, XGBoost expects us to provide the number of classes, and multiclass-enabled evaluation metrices.

With these parameters in place, we'll create a Ray AIR `XGBoostTrainer`:

In [5]:
# XGBoost specific params
params = {
    "tree_method": "approx",
    "objective": "multi:softmax",
    "eval_metric": ["mlogloss", "merror"],
    "num_class": 8,
}

trainer = XGBoostTrainer(
    scaling_config={"num_workers": 2},
    label_column="target",
    params=params,
    datasets={"train": train_dataset},
    num_boost_round=10,
)

The trainer contains a few default parameters. We can now create the Tuner with a search space for other parameters. During the run, it will then overwrite the default parameters in the trainer according to the search specification.

Here, we just want to tune the XGBoost `max_depth` parameter. Additionally, we want to compare our two `preprocessor` pipelines. Lastly, we configure Tune to minimize the `train-mlogloss` metric. In random search, this doesn't affect the evaluated configurations, but it will affect our default results fetching for analysis later.

In [6]:
tuner = Tuner(
    trainer,
    param_space={
        "params": {"max_depth": tune.randint(2, 8)},
        "preprocessor": tune.grid_search([prep_v1, prep_v2]),
    },
    tune_config=TuneConfig(num_samples=4, metric="train-mlogloss", mode="min"),
)

Let's run the tuning. This will take a few minutes to complete.

In [7]:
results = tuner.fit()

Trial name,status,loc,params/max_depth,preprocessor,iter,total time (s),train-mlogloss,train-merror
XGBoostTrainer_4544b_00000,TERMINATED,127.0.0.1:57483,7,"StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)",10,170.091,0.559286,0.197125
XGBoostTrainer_4544b_00001,TERMINATED,127.0.0.1:57490,3,"StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)",10,124.795,0.742797,0.279679
XGBoostTrainer_4544b_00002,TERMINATED,127.0.0.1:57492,4,"StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)",10,118.53,0.69843,0.262468
XGBoostTrainer_4544b_00003,TERMINATED,127.0.0.1:57493,7,"StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)",10,176.61,0.559286,0.197125
XGBoostTrainer_4544b_00004,TERMINATED,127.0.0.1:57494,2,"StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)",10,123.421,0.79498,0.311035
XGBoostTrainer_4544b_00005,TERMINATED,127.0.0.1:57679,2,"StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)",10,73.3928,0.79498,0.311035
XGBoostTrainer_4544b_00006,TERMINATED,127.0.0.1:57695,2,"StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)",10,67.2584,0.79498,0.311035
XGBoostTrainer_4544b_00007,TERMINATED,127.0.0.1:57698,6,"StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)",10,104.358,0.607805,0.2

(GBDTTrainable pid=57483) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(GBDTTrainable pid=57492) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(GBDTTrainable pid=57483) 2022-05-09 17:25:49,733	INFO main.py:984 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(GBDTTrainable pid=57492) 2022-05-09 17:25:50,325	INFO main.py:984 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(GBDTTrainable pid=57493) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(GBDTTrainable pid=57493) 2022-05-09 17:25:57,753	INFO main.py:984 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready 

Trial XGBoostTrainer_4544b_00002 reported train-mlogloss=1.52 with parameters={'params': {'max_depth': 4}, 'preprocessor': StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)}.
Trial XGBoostTrainer_4544b_00000 reported train-mlogloss=1.45 with parameters={'params': {'max_depth': 7}, 'preprocessor': StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)}.
Trial XGBoostTrainer_4544b_00002 reported train-mlogloss=1.27 with parameters={'params': {'max_depth': 4}, 'preprocessor': StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)}.


(GBDTTrainable pid=57494) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(GBDTTrainable pid=57490) 2022-05-09 17:26:20,736	INFO main.py:1029 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=57597) [17:26:20] task [xgboost.ray]:4493606160 got new rank 1
(_RemoteRayXGBoostActor pid=57596) [17:26:20] task [xgboost.ray]:4600864528 got new rank 0
(GBDTTrainable pid=57494) 2022-05-09 17:26:22,823	INFO main.py:984 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(raylet) Spilled 4345 MiB, 76 objects, write throughput 349 MiB/s.


Trial XGBoostTrainer_4544b_00003 reported train-mlogloss=1.45 with parameters={'params': {'max_depth': 7}, 'preprocessor': StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)}.
Trial XGBoostTrainer_4544b_00002 reported train-mlogloss=1.10 with parameters={'params': {'max_depth': 4}, 'preprocessor': StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness

...

(GBDTTrainable pid=57698) 2022-05-09 17:29:31,184	INFO main.py:1113 -- Training in progress (61 seconds since last restart).


Trial XGBoostTrainer_4544b_00007 reported train-mlogloss=0.63 with parameters={'params': {'max_depth': 6}, 'preprocessor': StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)}.
Trial XGBoostTrainer_4544b_00007 reported train-mlogloss=0.61 with parameters={'params': {'max_depth': 6}, 'preprocessor': StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Ty

(GBDTTrainable pid=57698) 2022-05-09 17:29:40,706	INFO main.py:1526 -- [RayXGBoost] Finished XGBoost training on training data with total N=581,012 in 84.95 seconds (70.52 pure XGBoost training time).
2022-05-09 17:29:41,178	INFO logger.py:730 -- Removed the following hyperparameter values when logging to tensorboard: {'preprocessor': StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, 

Trial XGBoostTrainer_4544b_00007 completed. Last result: train-mlogloss=0.607805,train-merror=0.220687,should_checkpoint=True


2022-05-09 17:29:41,405	INFO tune.py:753 -- Total run time: 246.90 seconds (245.40 seconds for the tuning loop).


Now that we obtained the results, we can analyze them. For instance, we can fetch the best observed result according to the configured `metric` and `mode` and print it:

In [8]:
# This will fetch the best result according to the `metric` and `mode` specified
# in the `TuneConfig` above:

best_result = results.get_best_result()

print("Best result error rate", best_result.metrics["train-merror"])

Best result error rate 0.197125


For more sophisticated analysis, we can get a pandas dataframe with all trial results:

In [9]:
df = results.get_dataframe()
print(df.columns)

Index(['train-mlogloss', 'train-merror', 'time_this_iter_s',
       'should_checkpoint', 'done', 'timesteps_total', 'episodes_total',
       'training_iteration', 'trial_id', 'experiment_id', 'date', 'timestamp',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'timesteps_since_restore', 'iterations_since_restore', 'warmup_time',
       'config/params', 'config/preprocessor', 'logdir'],
      dtype='object')


As an example, let's group the results per preprocessor and fetch the minimal obtained values:

In [10]:
groups = df.groupby("config/preprocessor")
mins = groups.min()

print("Preprocessor 1 minimum log loss:", mins.loc[str(prep_v1)]["train-mlogloss"])
print("Preprocessor 2 minimum log loss:", mins.loc[str(prep_v2)]["train-mlogloss"])

print("Preprocessor 1 minimum error:", mins.loc[str(prep_v1)]["train-merror"])
print("Preprocessor 2 minimum error:", mins.loc[str(prep_v2)]["train-merror"])

Preprocessor 1 minimum log loss: 0.559286
Preprocessor 2 minimum log loss: 0.559286
Preprocessor 1 minimum error: 0.197125
Preprocessor 2 minimum error: 0.197125


The `results.get_dataframe()` returns the last reported results per trial. If you want to obtain the best _ever_ observed results, you can pass the `filter_metric` and `filter_mode` arguments to `results.get_dataframe()`. In our example, we'll filter the minimum _ever_ observed `train-merror` for each trial:

In [11]:
df_min_error = results.get_dataframe(filter_metric="train-merror", filter_mode="min")
df_min_error["train-merror"]

0    0.197125
1    0.279679
2    0.262468
3    0.197125
4    0.310307
5    0.310307
6    0.310307
7    0.220687
Name: train-merror, dtype: float64

And that's how you analyze your hyperparameter tuning results. If you would like to have access to more analytics, please feel free to file a feature request e.g. [as a Github issue](https://github.com/ray-project/ray/issues) or on our [Discuss platform](https://discuss.ray.io/)!